In [1]:
# 49035114000 (center)
# 49035980000 (airport)
# 49035110106 (ski)
# 49035101402 (U of U)

In [1]:
# ============================================================
# FAST VERSION
# Build OD-level annual statistics for Complete Trips
# OD defined strictly by Census Tract (GEOID + shapefile)
# Population-level statistics (NOT sampled)
# ============================================================

import pandas as pd
import numpy as np
import geopandas as gpd
import glob
import json
from datetime import datetime
import pygeohash as pgh

# =========================
# CONFIG
# =========================

BASE_DIR = "C:/Users/rli04/Villanova University/Complete-trip-coordinate - Documents/General"
PARQUET_DIR = f"{BASE_DIR}/Salt_Lake/delivery"

TRACT_SHP = (
    f"{BASE_DIR}/Manuscript/Figure/Visualization-RL/"
    f"2-OD patterns by census track/six_counties_track.shp"
)

ORIG_TRACT = "49035114000"
DEST_TRACT = "49035980000"
# 49035114000 (center)
# 49035980000 (airport)
# 49035110106 (ski)
# 49035101402 (U of U)
MONTHS = [
    "Jan","Feb","Mar","Apr","May","Jun",
    "Jul","Aug","Sep","Oct","Nov","Dec"
]

OUTPUT_STATS_JSON = f"{ORIG_TRACT}_to_{DEST_TRACT}.stats.json"

USE_COLS = [
    "linked_trip_id",
    "travel_mode",
    "local_datetime_start",
    "local_datetime_end",
    "geohash7_orig",
    "geohash7_dest"
]

# =========================
# 0️⃣ LOAD TRACTS
# =========================

tracts = gpd.read_file(TRACT_SHP).to_crs("EPSG:4326")
tracts["GEOID"] = tracts["GEOID"].astype(str)

orig_geom = tracts.loc[tracts["GEOID"] == ORIG_TRACT, "geometry"].iloc[0]
dest_geom = tracts.loc[tracts["GEOID"] == DEST_TRACT, "geometry"].iloc[0]

# =========================
# 1️⃣ LOAD YEARLY DATA
# =========================

dfs = []

for m in MONTHS:
    files = glob.glob(f"{PARQUET_DIR}/Salt_Lake-{m}-2020/*.snappy.parquet")
    if not files:
        continue
    dfs.append(pd.concat(
        [pd.read_parquet(f, columns=USE_COLS) for f in files],
        ignore_index=True
    ))

df = pd.concat(dfs, ignore_index=True)

df["local_datetime_start"] = pd.to_datetime(df["local_datetime_start"], errors="coerce")
df["local_datetime_end"]   = pd.to_datetime(df["local_datetime_end"], errors="coerce")

df = df[df["local_datetime_end"] > df["local_datetime_start"]]

# =========================
# 2️⃣ VECTORIZE GEOHASH → POINT
# =========================

lat_o, lon_o = zip(*df["geohash7_orig"].map(pgh.decode))
lat_d, lon_d = zip(*df["geohash7_dest"].map(pgh.decode))

df["orig_lon"] = lon_o
df["orig_lat"] = lat_o
df["dest_lon"] = lon_d
df["dest_lat"] = lat_d

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.orig_lon, df.orig_lat),
    crs="EPSG:4326"
)

# =========================
# 3️⃣ OD FILTER (SPATIAL JOIN, FAST)
# =========================

gdf = gpd.sjoin(
    gdf,
    tracts.loc[tracts["GEOID"] == ORIG_TRACT][["geometry"]],
    predicate="within",
    how="inner"
)

gdf = gpd.GeoDataFrame(
    gdf.drop(columns="geometry"),
    geometry=gpd.points_from_xy(gdf.dest_lon, gdf.dest_lat),
    crs="EPSG:4326"
)

gdf = gpd.sjoin(
    gdf,
    tracts.loc[tracts["GEOID"] == DEST_TRACT][["geometry"]],
    predicate="within",
    how="inner"
)

# =========================
# 4️⃣ COMPLETE TRIP FILTER (≥2 legs)
# =========================

leg_counts = gdf.groupby("linked_trip_id").size()
valid_ids = leg_counts[leg_counts >= 2].index
gdf = gdf[gdf["linked_trip_id"].isin(valid_ids)]

# =========================
# 5️⃣ LEG DURATION (VECTOR)
# =========================

gdf["duration_min"] = (
    gdf["local_datetime_end"] - gdf["local_datetime_start"]
).dt.total_seconds() / 60

# =========================
# 6️⃣ COMPLETE-TRIP AGGREGATION (FAST)
# =========================

trip_stats = (
    gdf
    .groupby("linked_trip_id")
    .agg(
        total_duration=("duration_min", "sum"),
        transfers=("duration_min", "size"),
        modes=("travel_mode", lambda x: set(m.lower().strip() for m in x))
    )
)

trip_stats["transfers"] -= 1

# =========================
# 7️⃣ FINAL STATS
# =========================

dur = trip_stats["total_duration"].values
trf = trip_stats["transfers"].values
modes = trip_stats["modes"].values

def pct(a, q): return float(np.percentile(a, q))

stats = {
    "schema": "nova.complete_trip.od_stats.v1",
    "generated_at": datetime.utcnow().isoformat() + "Z",
    "od": {"origin": ORIG_TRACT, "destination": DEST_TRACT},
    "coverage": {"temporal": "year-2020", "spatial": "Salt Lake 6-county"},
    "counts": {"linked_trips": int(len(dur))},
    "trip_duration_min": {
        "min": float(dur.min()),
        "p25": pct(dur, 25),
        "median": pct(dur, 50),
        "p75": pct(dur, 75),
        "max": float(dur.max())
    },
    "transfers": {
        "avg": float(trf.mean()),
        "p75": int(pct(trf, 75)),
        "max": int(trf.max())
    },
    "mode_involvement": {
        "car": float(sum("car" in m for m in modes) / len(modes)),
        "bus": float(sum("bus" in m for m in modes) / len(modes)),
        "rail": float(sum("rail" in m for m in modes) / len(modes)),
        "walk": float(sum("walk" in m for m in modes) / len(modes))
    }
}

with open(OUTPUT_STATS_JSON, "w", encoding="utf-8") as f:
    json.dump(stats, f, indent=2, allow_nan=False)

print("Stats JSON written to:", OUTPUT_STATS_JSON)


ValueError: 'index_left' and 'index_right' cannot be names in the frames being joined